# Prepare Cells with Dilute Mixing
Generate initial structures where we aliovalently dope each perovskite. 

In [23]:
from collections import defaultdict
from ase.db import connect
import pandas as pd

Configuration

In [24]:
supercell: int = 2  # Number of unit cells per simulation cell

## Load the Initial Structures
Get the Atoms object for each structure and the charge states of atoms on each site.

In [25]:
pure_structures = []
with connect('atoms-relax.db') as db:
    for row in db.select(supercell=supercell):
        atoms = row.toatoms(True)
        pure_structures.append({
            **atoms.info['key_value_pairs'],
            'atoms': atoms,
        })
pure_structures = pd.DataFrame(pure_structures)
pure_structures.head()

,name,a,b,a_val,b_val,supercell,atoms
0,CeAlO3,Ce,Al,3,3,2,"(Atom('Ce', [0.0, 0.0, 0.0], magmom=7.0, index..."
1,TbAlO3,Tb,Al,3,3,2,"(Atom('Tb', [-1.918220590456243e-06, 1.2258864..."
2,MnGeO3,Mn,Ge,2,4,2,"(Atom('Mn', [-1.9866617139540923e-06, -1.29286..."
3,YScO3,Y,Sc,3,3,2,"(Atom('Y', [-2.6041382337886296e-05, 5.5206279..."


Remove structures with unknown valency

In [26]:
pure_structures.query('a_val > 0', inplace=True)
print(f'Found a total of {len(pure_structures)} starting points')

Found a total of 4 starting points


Assemble atoms by allowed valency

In [27]:
elements_by_val = defaultdict(set)
for l in ['a', 'b']:
    for gid, group in pure_structures.groupby(f'{l}_val'):
        elements_by_val[gid].update(group[l].tolist())

In [28]:
for ox, count in elements_by_val.items():
    print(f'{len(count)} elements in {ox}+')

1 elements in 2+
5 elements in 3+
1 elements in 4+


## Assemble the Dilute Mixing Structures
Replace the first atom of a certain type with each of the the atoms which can take on the same valency

In [29]:
with connect('initial-dilute-mixing.db') as db:
    for rid, row in pure_structures.iterrows():  # Over all pure structures
        for l in ['a', 'b']:
            # Find the atom to replace
            old_elem = row[l]
            old_id = row['atoms'].symbols.index(old_elem)

            # Iterate over all possible replacements
            for new_elem in elements_by_val[row[f'{l}_val']]:
                new_atoms = row['atoms'].copy()
                if new_elem in row[['a', 'b']].tolist():
                    continue

                # Make a new name
                site_count = new_atoms.get_chemical_symbols().count(old_elem)
                if l == 'a':
                    name = f'({old_elem}{site_count - 1}{new_elem}1){row["b"]}O3'
                else:
                    name = f'{row["a"]}({old_elem}{site_count - 1}{new_elem}1)O3'

                # Alter the structure
                new_symbols = new_atoms.get_chemical_symbols()
                new_symbols[old_id] = new_elem
                new_atoms.set_chemical_symbols(new_symbols)

                # Add to the db if it doesn't exist
                if db.count(name=name, supercell=supercell) == 0:
                    db.write(new_atoms, name=name, dopant=new_elem, site=l, **row.drop(['atoms', 'name']))